In [ ]:
# Articles
# - https://jyx.jyu.fi/handle/123456789/78083

In [ ]:
# Notes

# Important paper  => Kania2021_Chapter_InteractiveMultiobjectiveOptim.pdf

In [ ]:
# 
"""
Toimitusaika = 12kk
Tilauskustannus = 100 + 100

Ketjutettutilaus eteenpäin


Liquid zone -> 
Frozen zone -> määritelty materiaali tulee, oli tilattu tai ei

Vuoden jaksotettut tilaukset -> Milloin tilataan ja paljon
"""

In [19]:
import pandas as pd
import numpy as np

df = pd.read_excel('Demand_data_case.xlsx')

months = []
month = 1
for i in range(1, 109):
    if month % 13 == 0:
        month = 1
    months.append(month)
    month = month + 1
df['months'] = months
df

print(df)

x1 = np.array(df['Demand (pcs)'])  # Demand
x2 = np.array(df['months'])        # Months / timeline
x3 = np.array(df['Demand (pcs)'])  # Orders

x = pd.array([x1, x2, x3])

     Month  Demand (pcs)  months
0    m_001            11       1
1    m_002            12       2
2    m_003            10       3
3    m_004             8       4
4    m_005            12       5
..     ...           ...     ...
103  m_104             3       8
104  m_105             0       9
105  m_106             0      10
106  m_107             3      11
107  m_108             5      12

[108 rows x 3 columns]


In [20]:
# Objectives
"""
f1 -> Availability / No stock out -> service level / fill rate
    Maximize
    Total Number of Customer Orders Shipped / Number of Customer Orders Filled

f2 -> Holding cost
    Minimize

    Q = Optimal order quantity
    H = Holding cost per unit => Fixed 10% of unit price
    Q/2 x H
f3 -> Inventory turnover rate # Not part of this project
"""


def f1(x):
    return np.nan_to_num(x[0] / x[2])

def f2(x):
    return (x[2] / 2) * 6500*0.1

f_1 = f1(x)
f_2 = f2(x)

print(f_1)
print(f_2)


[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 1.
 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1.
 1. 0. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1.
 1. 1. 0. 1. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 1. 1.]
[3575. 3900. 3250. 2600. 3900. 2925. 3900.  975. 1625. 1950. 1300. 2925.
 3250.    0.    0.    0.    0. 1300. 1950. 3900. 2925. 3900.    0. 1950.
 5200. 7800. 2600. 5525.  975.  650.    0.    0.  650. 1300. 3250. 2275.
 3250.  650. 1625. 1625. 3900.    0. 2925.  650.    0. 2600.  650.  975.
 5200.    0. 3900. 1300. 1300.  650. 1950.    0.    0. 1300. 1300. 1300.
 3900. 3900. 5850.  650. 2925. 1300. 2600.    0.  975.  650.    0. 1300.
 2600. 1950.    0. 3575. 2275. 1300. 4550.    0.    0.    0.  650.  975.
 5200.  650. 2600. 3250.    0. 1950.    0.    0.    0.    0.  650.    0.
    0.    0.    0. 1625. 3575. 1625. 1300.  975.    0.    0.  975. 1625.]


/var/folders/xp/ptg6qcks2j72m9d_slq4zjs80000gn/T/ipykernel_45744/3943425547.py:18: RuntimeWarning: invalid value encountered in true_divide
  return np.nan_to_num(x[0] / x[2])


In [57]:
# Constraits
"""
c1 -> Stock greater than zero
c2 -> Max stock amount
"""
from desdeo_problem import ScalarConstraint

def const_func_1(x, y):
    ret = []
    for yi in y:
        ret.append(1 - yi[0])
    return np.array(ret)

c1 = ScalarConstraint('c_1', 3, 2, const_func_1)

# def const_func_2(x, y):
#     ret = []
#     for yi in y:
#         ret.append(1 - yi[0])
#     return np.array(ret)

# c2 = ScalarConstraint('c_1', 3, 2, const_func_2)

In [3]:

# Safety stock

R = np.mean(x[0])
std_R = np.std(x[0])
L = 12 # lead time - 12 months
#sL = 0 # lead time standard deviation
CSL = 0.9 # cycle service level

SS = (CSL * std_R * L)**0.5
print('safety stock', SS)

print('Demand mean / average', R)
print('Demand standard deviation', std_R)

safety stock 7.410282970677182
Demand mean / average 5.333333333333333
Demand standard deviation 5.084471639398911


In [22]:
# Surrogate
from desdeo_problem import VectorDataObjective as VDO
from desdeo_problem.surrogatemodels.SurrogateModels import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern

#data = np.hstack((x, f_1, f_2))
#print(data)

data = {
    'x1': x1,  # demand
    'x2': x2,  # months
    'x3': x3,  # orders
    'f1': f_1, 
    'f2': f_2
}

datapd = pd.DataFrame(data=data) # columns=['x1', 'x2', 'f1', 'f2']

print(datapd)
obj = VDO(data=datapd, name=['f1', 'f2'])

obj.train(models=GaussianProcessRegressor, model_parameters={'kernel': Matern(nu=1.5)})
 
print(obj.evaluate(np.asarray([[3,2, 4]]), use_surrogate=True))

     x1  x2  x3   f1      f2
0    11   1  11  1.0  3575.0
1    12   2  12  1.0  3900.0
2    10   3  10  1.0  3250.0
3     8   4   8  1.0  2600.0
4    12   5  12  1.0  3900.0
..   ..  ..  ..  ...     ...
103   3   8   3  1.0   975.0
104   0   9   0  0.0     0.0
105   0  10   0  0.0     0.0
106   3  11   3  1.0   975.0
107   5  12   5  1.0  1625.0

[108 rows x 5 columns]
Objective Evaluation Results Object 
Objective values are: 
         f1           f2
0  1.086917  1137.356045
Uncertainity values are: 
         f1        f2
0  0.080799  0.098621



/usr/local/lib/python3.9/site-packages/sklearn/gaussian_process/_gpr.py:610: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
/usr/local/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but GaussianProcessRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but GaussianProcessRegressor was fitted with feature names
  warnings.warn(


In [58]:
# Problem

from desdeo_problem import DataProblem

maximize = pd.DataFrame([[True, False]], columns=['f1','f2'])
prob = DataProblem(data=datapd, objective_names=['f1','f2'], variable_names=['x1', 'x2', 'x3'], constraints=[c1], maximize=maximize)

prob.train(GaussianProcessRegressor)

data_to_evaluate = []
for i in range(len(x1)):
    data_to_evaluate.append([x1[i], x2[i], x3[i]])


print(prob.evaluate(np.asarray(data_to_evaluate), use_surrogate=True))

Evaluation Results Object 
Objective values are: 
[[ 1.00000000e+00  3.57500000e+03]
 [ 1.00000000e+00  3.90000000e+03]
 [ 1.00000000e+00  3.25000000e+03]
 [ 1.00000000e+00  2.60000000e+03]
 [ 1.00000000e+00  3.90000000e+03]
 [ 1.00000000e+00  2.92500000e+03]
 [ 1.00000000e+00  3.90000000e+03]
 [ 1.00000000e+00  9.75000000e+02]
 [ 1.00000000e+00  1.62500000e+03]
 [ 1.00000000e+00  1.95000000e+03]
 [ 1.00000000e+00  1.30000000e+03]
 [ 1.00000000e+00  2.92500000e+03]
 [ 1.00000000e+00  3.25000000e+03]
 [ 5.53237840e-13  3.62525610e-10]
 [ 1.08987567e-18  3.00132791e-17]
 [ 1.22771020e-12  7.13249239e-10]
 [ 2.77426453e-15  3.08173523e-12]
 [ 1.00000000e+00  1.30000000e+03]
 [ 1.00000000e+00  1.95000000e+03]
 [ 1.00000000e+00  3.90000000e+03]
 [ 1.00000000e+00  2.92500000e+03]
 [ 1.00000000e+00  3.90000000e+03]
 [ 5.55551766e-13  2.78453088e-10]
 [ 1.00000000e+00  1.95000000e+03]
 [ 1.00000000e+00  5.20000000e+03]
 [ 1.00000000e+00  7.80000000e+03]
 [ 1.00000000e+00  2.60000000e+03]
 [ 1.

/usr/local/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but GaussianProcessRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but GaussianProcessRegressor was fitted with feature names
  warnings.warn(
